In [3]:
import numpy as np
import torch
import pandas as pd
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# table = pd.read_csv(os.path.join("/workspace/G_Project/weights/valence/TEST/1s/128e4/fold_1", "output.csv"))['v_CE']

In [80]:
# target = "arousal"
# length = "60s"
# batch_size = "128"

# for lr in ["e3","e4","e5"]:
#     indices = []
#     for fold in ["fold_1","fold_2","fold_3","fold_4","fold_5"]:
#         path = os.path.join("/workspace/G_Project/weights", target, "TEST", length, batch_size+lr, fold, "output.csv")
#         rmse = pd.read_csv(path)["v_RMSE"] # 기입 위치 잘못됨
#         if np.isnan(rmse[0]):
#             rmse = pd.read_csv(path)['v_CE']
#         indices.append(np.argmin(rmse))

#     print("===============================")
#     print(f'- Batch: {batch_size}')
#     print(f'- LR: {lr}')
#     print(f'- Stop Epochs: {indices}')

- Batch: 128
- LR: e3
- Stop Epochs: [11, 13, 4, 18, 0]
- Batch: 128
- LR: e4
- Stop Epochs: [65, 1, 17, 19, 0]
- Batch: 128
- LR: e5
- Stop Epochs: [1, 9, 12, 142, 2]


In [4]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive',
]
json_file_name = 'graduation-project-399003.json'

In [5]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)

In [6]:
gc = gspread.authorize(credentials)

In [7]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1-caaREBN1rI1TT_xe6p21zC0-NGDOlGZ23HSZPPCdNc/edit#gid=930989279'

In [8]:
doc = gc.open_by_url(spreadsheet_url)

In [9]:
worksheet = doc.worksheet('Full Length')

In [11]:
worksheet.acell('CV_RMSE').value

IncorrectCellLabel: CV_RMSE

In [25]:
print(worksheet.acell('G8').value)

None


In [26]:
worksheet.update_acell('G8', 10)

{'spreadsheetId': '1-caaREBN1rI1TT_xe6p21zC0-NGDOlGZ23HSZPPCdNc',
 'updatedRange': "'Full Length'!G8",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}